worked after meeting with Maikel to change format to lastupdate, validto, volume_Type1, Volume_type2, etc. 
using pivot function... issue is that last update values are unique to Type, resulting in many empty cells, typically, for all other Types than the Type that was updated that exact second.... 

I'll look again when I have time, but if any of you has time to look into this, great! be careful, code to add it to WARP.db is commented out, so review the df direclty instead (data wrangler recommended)
Twan, 9 May 22:00 CET

Twan 10 May 6:25: allmost have a solution!!! just need to make the current_datetime (time of fetch) less granular... that way to collade. will do so tomorrow!

 group current_datetime values that are within a 1 minute range, and change all in the group to the most recent current_datetime value in each group, as definition for a new variable 'fetch_moment'

In [ ]:
import pandas as pd
import sqlite3
import pandas as pd


# Connect to the SQLite database
db_path = '../data/WARP.db'
conn = sqlite3.connect(db_path)

# Read the raw_NED_preds table into a DataFrame
df_NED_preds_processed = pd.read_sql_query("SELECT * FROM raw_NED_preds", conn)

# Close the database connection
conn.close()


import pandas as pd

# Zet datumkolommen om naar datetime
df_NED_preds_processed['validfrom'] = pd.to_datetime(df_NED_preds_processed['validfrom'])
df_NED_preds_processed['validto'] = pd.to_datetime(df_NED_preds_processed['validto'])
df_NED_preds_processed['lastupdate'] = pd.to_datetime(df_NED_preds_processed['lastupdate'])
df_NED_preds_processed['current_datetime'] = pd.to_datetime(df_NED_preds_processed['current_datetime'])

# Zet 'lastupdate' als index en zorg dat deze ook datetime is
#df_NED_preds_processed.set_index('lastupdate', inplace=True)
#df_NED_preds_processed.index = pd.to_datetime(df_NED_preds_processed.index)



# Calculate rows where validfrom is earlier than lastupdate
mask = df_NED_preds_processed['validfrom'] >= df_NED_preds_processed['lastupdate']
# Keep only rows that match this condition
df_NED_preds_processed = df_NED_preds_processed[mask]
print(f"Number of rows where validfrom is earlier than lastupdate: {(~mask).sum()}")
# Sort by current_datetime to ensure correct grouping
df_NED_preds_processed = df_NED_preds_processed.sort_values('current_datetime')


# dealing with fact that current_datetime is not always the same across Types fetched 'simultaneously':
# Group current_datetime values within 1 minute and assign the most recent (max) value in each group as 'fetch_moment'
df_NED_preds_processed['fetch_moment'] = (
    df_NED_preds_processed['current_datetime']
    .diff().gt(pd.Timedelta('1min')).cumsum()
)
# Map each group to its max current_datetime
fetch_moment_map = df_NED_preds_processed.groupby('fetch_moment')['current_datetime'].transform('max')
df_NED_preds_processed['fetch_moment'] = fetch_moment_map

# Verwijder irrelevante kolommen
df_NED_preds_processed = df_NED_preds_processed.drop(columns=[
    '@id', '@type', 'id', 'point', 'granularity', 'granularitytimezone', 'activity', 'classification', 'capacity','percentage','emission','emissionfactor','validfrom','lastupdate'])


# Verander de naam van de kolom 'type' naar 'PRED_Type'
#df_NED_preds_processed['type'] = df_NED_preds_processed['type'].str.replace('/v1/types/', 'PRED_Type_')
#print("Unique values in 'type':", df_NED_preds_processed['type'].unique())

# Sort the DataFrame by the index,  by 'validto', asnd by 'type'

df_NED_preds_processed = df_NED_preds_processed.sort_values(by=['fetch_moment','validto', 'type'])
print(df_NED_preds_processed.head(20))



Number of rows where validfrom is earlier than lastupdate: 8677
               type    volume                   validto    current_datetime  \
1463    /v1/types/1   2141122 2025-04-02 09:00:00+00:00 2025-04-02 08:17:40   
29823  /v1/types/17   2799828 2025-04-02 09:00:00+00:00 2025-04-02 08:17:41   
30373   /v1/types/2  11265053 2025-04-02 09:00:00+00:00 2025-04-02 08:17:41   
1997   /v1/types/20    485000 2025-04-02 09:00:00+00:00 2025-04-02 08:17:42   
11076  /v1/types/21    365000 2025-04-02 09:00:00+00:00 2025-04-02 08:17:43   
36932  /v1/types/26    221000 2025-04-02 09:00:00+00:00 2025-04-02 08:17:43   
1464    /v1/types/1   2219175 2025-04-02 10:00:00+00:00 2025-04-02 08:17:40   
29824  /v1/types/17   2716766 2025-04-02 10:00:00+00:00 2025-04-02 08:17:41   
30374   /v1/types/2  14442681 2025-04-02 10:00:00+00:00 2025-04-02 08:17:41   
1998   /v1/types/20    485000 2025-04-02 10:00:00+00:00 2025-04-02 08:17:42   
11077  /v1/types/21    365000 2025-04-02 10:00:00+00:00 2025-04-02 

In [4]:
# Pivot the table
pivoted = df_NED_preds_processed.pivot_table(
    index=['fetch_moment', 'validto'],  # keep these as index
    columns='type',                   # columns become unique values from 'type'
    values='volume',                  # values to fill in the new columns
    aggfunc='first'                   # if duplicates exist, take the first
)

# Optional: flatten column names and rename to match desired output
pivoted.columns = [f'Volume_{col}' for col in pivoted.columns]


# To avoid tuple-as-single-column issue:
pivoted = pivoted.reset_index()
print(pivoted.head())

# Save the processed DataFrame to a new SQLite database
output_db_path = '../data/WARP.db'
conn = sqlite3.connect(output_db_path)
pivoted.to_sql('processed_NED_preds', conn, if_exists='replace', index=True)
conn.close()

         fetch_moment                   validto  Volume_/v1/types/1  \
0 2025-04-02 08:17:43 2025-04-02 09:00:00+00:00           2141122.0   
1 2025-04-02 08:17:43 2025-04-02 10:00:00+00:00           2219175.0   
2 2025-04-02 08:17:43 2025-04-02 11:00:00+00:00           2379837.0   
3 2025-04-02 08:17:43 2025-04-02 12:00:00+00:00           2802929.0   
4 2025-04-02 08:17:43 2025-04-02 13:00:00+00:00           3373530.0   

   Volume_/v1/types/17  Volume_/v1/types/2  Volume_/v1/types/20  \
0            2799828.0          11265053.0             485000.0   
1            2716766.0          14442681.0             485000.0   
2            2775487.0          17357228.0             485000.0   
3            2580169.0          18601126.0             485000.0   
4            2731686.0          18169871.0             485000.0   

   Volume_/v1/types/21  Volume_/v1/types/26  Volume_/v1/types/59  
0             365000.0             221000.0                  NaN  
1             365000.0             2

In [ ]:
'''# dealing with NaN values, specifically those cells with numberical values in (both) adjacent cells
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# First make a copy to avoid modifying the original dataframe
df_filled = df_wide.copy()

# Ensure index is datetime for time-based interpolation
df_filled.index = pd.to_datetime(df_filled.index)

# For each column in the dataframe
for column in df_filled.columns:
    # Forward and backward fill to get values for interpolation
    series = df_filled[column]
    
    # Get the index as datetime for hourly interpolation
    index_as_datetime = pd.to_datetime(series.index)
    
    # Only interpolate if there's a value both before and after (linear interpolation)
    df_filled[column] = series.interpolate(
        method='time',
        limit=1, # limit=1 ensures we only interpolate for consecutive hours
        limit_direction='both' # limit_direction='both' ensures we only interpolate between actual values
    )

# Print some statistics about the filling
for column in df_filled.columns:
    filled_count = df_filled[column].count() - df_wide[column].count()
    if filled_count > 0:
        print(f"Filled {filled_count} NaN values in {column}")

Filled 27 NaN values in volume_pred_type_17_d0
Filled 1 NaN values in volume_pred_type_17_d1
Filled 1 NaN values in volume_pred_type_17_d2
Filled 1 NaN values in volume_pred_type_17_d3
Filled 1 NaN values in volume_pred_type_17_d4
Filled 1 NaN values in volume_pred_type_17_d5
Filled 1 NaN values in volume_pred_type_17_d6
Filled 38 NaN values in volume_pred_type_17_d7
Filled 27 NaN values in volume_pred_type_1_d0
Filled 1 NaN values in volume_pred_type_1_d1
Filled 1 NaN values in volume_pred_type_1_d2
Filled 1 NaN values in volume_pred_type_1_d3
Filled 1 NaN values in volume_pred_type_1_d4
Filled 1 NaN values in volume_pred_type_1_d5
Filled 1 NaN values in volume_pred_type_1_d6
Filled 38 NaN values in volume_pred_type_1_d7
Filled 27 NaN values in volume_pred_type_20_d0
Filled 1 NaN values in volume_pred_type_20_d1
Filled 1 NaN values in volume_pred_type_20_d2
Filled 1 NaN values in volume_pred_type_20_d3
Filled 1 NaN values in volume_pred_type_20_d4
Filled 1 NaN values in volume_pred_ty

In [ ]:
'''import sqlite3  

# Save the filled DataFrame to a new SQLite database
output_db_path = '../data/WARP.db'
conn = sqlite3.connect(output_db_path)  
df_filled.to_sql('processed_NED_preds', conn, if_exists='replace', index=True)
conn.close()
# Check the filled DataFrame
print(df_filled.head())

pred_column                volume_pred_type_17_d0  volume_pred_type_17_d1  \
validto                                                                     
2025-04-09 02:00:00+00:00               1264364.0               1078600.0   
2025-04-09 03:00:00+00:00               1181091.0                953443.0   
2025-04-09 04:00:00+00:00                899096.0                844335.0   
2025-04-09 05:00:00+00:00               1004562.0                807438.0   
2025-04-09 06:00:00+00:00               1551765.0                949137.0   

pred_column                volume_pred_type_17_d2  volume_pred_type_17_d3  \
validto                                                                     
2025-04-09 02:00:00+00:00               1105664.0                977644.0   
2025-04-09 03:00:00+00:00               1024192.0                862923.0   
2025-04-09 04:00:00+00:00               1000249.0                825005.0   
2025-04-09 05:00:00+00:00               1151357.0                973646.0  

In [ ]:
'''# Connect to the SQLite database and read the processed_NED_preds table, to verify the data
conn = sqlite3.connect('../data/WARP.db')
df = pd.read_sql_query("SELECT * FROM processed_NED_preds", conn)
conn.close()


print("DataFrame shape:", df.shape)
print("First five rows of the DataFrame:")
print(df.head())

DataFrame shape: (801, 57)
First five rows of the DataFrame:
                     validto  volume_pred_type_17_d0  volume_pred_type_17_d1  \
0  2025-04-09 02:00:00+00:00               1264364.0               1078600.0   
1  2025-04-09 03:00:00+00:00               1181091.0                953443.0   
2  2025-04-09 04:00:00+00:00                899096.0                844335.0   
3  2025-04-09 05:00:00+00:00               1004562.0                807438.0   
4  2025-04-09 06:00:00+00:00               1551765.0                949137.0   

   volume_pred_type_17_d2  volume_pred_type_17_d3  volume_pred_type_17_d4  \
0               1105664.0                977644.0               1294379.0   
1               1024192.0                862923.0               1302783.0   
2               1000249.0                825005.0               1350416.0   
3               1151357.0                973646.0               1535660.0   
4               1372373.0               1199476.0               1737786.0